# Naive Bayes

De Naive Bayes techniek is een **supervised** machine learning techniek voor **classificatie** problemen op te lossen.
Dit is een redelijke oude techniek en wordt tegenwoordig al iets minder gebruikt.
Echter is het een zeer snelle techniek, zelfs als er heel veel features zijn.
Hierdoor is deze techniek wel nog nuttig in real-time toepassingen.

Het verschil met de SVM of logistische regressie classifiers is dat dit een **Generative classifier** is in plaats van een Discriminatieve classifier.
Een discriminatieve classifier is een classifier die de grens leer tussen twee verschillende klassen.
Dit wil zeggen dat er een scheidingslijn gezocht wordt die geoptimaliseerd wordt op basis van misclassificaties.
Bij discriminative wordt er dus vooral gekeken naar wat de klassen verschillend maakt.
Een generative classifier echter gaat vooral kijken naar wat observaties van dezelfde klassen gemeenschappelijk hebben (welke features maken een observatie tot die klasse)
Hierbij worden de verschillende features bekeken en er wordt gezocht naar een algemene verdeling van de observaties die tot elke klasse hoort. 

De benaming komt voort uit het feit dat Naive Bayes gebruikt maakt van de Bayes regel over conditionele probabiliteiten waarbij gebruik gemaakt wordt van de naieve veronderstelling dat de features onafhankelijk zijn van elkaar.

## Werking: Bayes rule

**Voorbeeld:**

Kans dat je kanker hebt $1\%$ ($P(Kanker) = 1\% = 0.01$)

Stel dat er een test is om kanker op te sporen die:
* In 90% van de gevallen is de test positief als je kanker hebt (sensitiviteit)
* In 90% van de gevallen is de test negatief als je geen kanker hebt (specificiteit)

Wat is nu de kans dat je kanker hebt als deze test positief blijkt te zijn?

Deze kans is niet 90% omdat er ook rekening gehouden moet worden met de kans dat iemand kanker heeft.
We zijn dus echter op zoek naar $P(\text{Kanker}|\text{Positief})$.
Door gebruik te maken van de Bayes regel kan dit uitgeschreven worden als volgt:

$P(\text{Kanker}|\text{Positief}) = \frac{P(\text{Positief}|\text{Kanker})P(\text{Kanker})}{P(\text{Positief})}$

$P(\text{Kanker}|\text{Positief}) = \frac{0.9 * 0.01}{0.01 * 0.9 + 0.1*0.99} = 0.08333$

De verschillende kansen in deze uitdrukking hebben de volgende definities:
* Prior: $P(\text{Kanker})$: Kans dat iemand kanker heeft (zonder te testen)
* Likelihood: $P(\text{Positief}|\text{Kanker})$: De kans dat je positief test als je kanker hebt
* Marginal: $P(\text{Positief})$: De kans dat je positief test
* Posterior: $P(\text{Kanker}|\text{Positief})$: De kans dat je kanker hebt als je positief test

De algemene vorm van bovenstaande berekening kan geschreven worden als

$P(\text{H}|\text{e}) = \frac{P(\text{e}|\text{H})P(\text{H})}{P(\text{e})}$

waar: 
* e de evidence voorstelt (de features)
* H de hypothese voorstelt (de features horen bij klasse $i$)

Dit type classifier kan geimplementeerd worden door gebruik te maken van de [GaussianNB klasse](https://scikit-learn.org/stable/modules/naive_bayes.html) uit sklearn.

In [5]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

In [6]:
wine = datasets.load_wine()

X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.2)

naiveBayes = GaussianNB()
naiveBayes.fit(X_train, y_train)
y_pred = naiveBayes.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9444444444444444


# Tekstclassificatie

Een veel voorkomende toepassing bij machine learning is het werken op tekstfragmenten.
Denk bijvoorbeeld aan chatbots, spam-mail detectie, automatische call-centers, ...
Dit wordt ook NLP of Natural Language Processing genoemd.
De moeilijkheid bij het werken met NLP is dat de betekenis van een woord vaak sterk afhankelijk is van de context van het stuk tekst.
Om met deze complexiteit om te gaan zijn geavanceerde machine learning technieken nodig zoals deep learning.
Dit komt later bij Machine Learning in meer detail aan bod.
De complexiteit kan echter sterk gereduceerd worden door te veronderstellen dat alle woorden in het tekstfragment onafhankelijk zijn van elkaar.
In de rest van deze notebook wordt gewerkt met het typische voorbeeld over spam-detectie op inkomende mails.

## Typisch voorbeeld: Spam detectie

Hieronder staat een typisch voorbeeld van een spam mail.

![spam-example](images\spam1.jpg)

Het doel is nu om deze mail te classificeren als spam of not spam (dit laatste wordt ook "ham" genoemd).
We zoeken dus de kans dat deze tekst spam is op basis van alle woorden die erin staat.
Hierdoor bekomen we de volgende vergelijking.

$P(\text{Spam}|w_1,w_2, \dots w_n) = \frac{P(w_1,w_2, \dots w_n|\text{Spam}) P(\text{Spam})}{P(w_1,w_2, \dots w_n)}$

Na toepassing van de Bayes rule kunnen we dit ook schrijven als:

$P(\text{Spam}|w_1,w_2, \dots w_n) = \frac{P(w_1|w_2, \dots w_n,\text{Spam})P(w_2|w_3, \dots w_n,\text{Spam})\dots P(\text{Spam})}{P(w_1,w_2, \dots w_n)}$

De kansen in de teller van de breuk geven de kansen weer om een bepaald woord te vinden op basis van de andere woorden in de tekst.
Dit is heel lastig te berekenen en zorgt voor een heel hoge complexiteit.
Hier komt echter de veronderstelling van onafhankelijkheid van de woorden in de tekst goed van pas om deze vergelijking te vereenvoudigen.
We bekomen namelijk:

$P(\text{Spam}|w_1,w_2, \dots w_n) = \frac{P(w_1|\text{Spam})P(w_2|\text{Spam})\dots P(w_n|\text{Spam})P(\text{Spam})}{P(w_1,w_2, \dots w_n)} = \frac{P(\text{Spam})\prod \limits_{i=1}^{n}P(w_i|\text{Spam})}{P(w_1,w_2, \dots w_n)}$

De noemer hierin zorgt ervoor dat de teller terug omgezet wordt naar een kans (met een waarde tussen 0 en 1).
Deze is echter onafhankelijk van de kans of het spam is of niet.
We hebben ook niet de exacte kans nodig maar moeten gewoon weten of de teller het grootst is als het spam is of niet.
Daarom kan de noemer dus weggelaten worden met als volgende resultaat.

$P(\text{Spam}|w_1,w_2, \dots w_n) \propto P(\text{Spam})\prod \limits_{i=1}^{n}P(w_i|\text{Spam})$

Stel dat we de volgende gegevens hebben over een dataset met 300 spam mails en 850 ham mails:

In [1]:
import pandas as pd

In [2]:
aantal_spam_mails = 300
aantal_ham_mails = 850

woord = ["customer", "advise", "Africa", "money", "number"]
spam_freq = [100,50, 120,60, 180]
spam_prob = [x/aantal_spam_mails for x in spam_freq]
ham_freq = [200, 70, 30, 450, 660]
ham_prob =  [x/aantal_ham_mails for x in ham_freq]

df = pd.DataFrame({"Woord": woord, "Spam frequentie": spam_freq, "Spam kans": spam_prob, "Ham frequentie": ham_freq, "Ham kans": ham_prob})
df

,Woord,Spam frequentie,Spam kans,Ham frequentie,Ham kans
0,customer,100,0.333333,200,0.235294
1,advise,50,0.166667,70,0.082353
2,Africa,120,0.400000,30,0.035294
3,money,60,0.200000,450,0.529412
4,number,180,0.600000,660,0.776471


Beantwoord nu de volgende vragen:
* Wat is $P(\text{Spam})$, de kans dat een willekeurige mail spam is in de dataset?
* Doe een manuele classificatie van de zin "Africa advise money"

In [3]:
pSpam = 350/(850+300)
pHam = 1-pSpam
print(pSpam, pHam)

pTextIsSpam = pSpam * 0.4 * 0.17 * 0.2
pTextIsHam = pHam * 0.035 * 0.08 * 0.53

if(pTextIsSpam > pTextIsHam):
    print("Africa advise money is spam")
else:
    print("Africa advise money is ham")

0.30434782608695654 0.6956521739130435
Africa advise money is spam


**Hoe wordt er omgegaan met classificatie van woorden die niet in de dataset zaten?**

Indien een mail uit de testset een woord bevat die nooit gezien was in de trainingsset, dan kan deze mail niet geclassificeerd worden omdat de kans van het woord steeds 0 is.
Er zijn twee mogelijkheden om hiermee om te gaan. 
* Ofwel laat je die eruit vallen maar dan verlies je informatie.
* Ofwel geef je deze woorden toch een bepaalde kans.

Dit laatste kan gebeuren door middel van Laplacian Smoothing.
De formule hiervoor is: 

$P(w) = \frac{C(w) + \alpha}{N+\alpha V}$

met 
* P(w) de uiteindelijke kans van het woord
* C(w) het aantal mails waarin het woord voorkomt
* N het aantal spam-mails
* V het aantal verschillende woorden (features) in de dataset
* $\alpha$ een hyperparameter dat aangeeft hoeveel gewicht de ongeziene woorden moeten krijgen.
    * Kleine waarde $\rightarrow$ klein belang van ongeziene woorden $\rightarrow$ neiging tot overfitting
    * Grote waarde $\rightarrow$ groot belang van ongeziene woorden $\rightarrow$ neiging tot underfitting 
    
Het algemene concept van Laplacian smoothing is om voor ongeziene woorden een extra fictieve mail toe te voegen die enkel bestaat uit dit woord.
De $\alpha$ is dan hoeveel keer deze mail wordt toegevoegd.
Bereken voor een $\alpha=2$ opnieuw de kansen in het dataframe df.
Voeg hier ook een lijn aan toe voor woorden die niet in de dataset aanwezig zijn.

In [4]:
# bereken de geupdate matrix met de kansen

alpha=2
N_spam = 300
N_ham = 850
V =len(df)

df['KansSpamSmoothed'] = (df['Spam frequentie'] + alpha) / (N_spam + alpha * V)
df['KansHamSmoothed'] = (df['Ham frequentie'] + alpha) / (N_ham + alpha * V)

display(df)

,Woord,Spam frequentie,Spam kans,Ham frequentie,Ham kans,KansSpamSmoothed,KansHamSmoothed
0,customer,100,0.333333,200,0.235294,0.329032,0.234884
1,advise,50,0.166667,70,0.082353,0.167742,0.083721
2,Africa,120,0.400000,30,0.035294,0.393548,0.037209
3,money,60,0.200000,450,0.529412,0.200000,0.525581
4,number,180,0.600000,660,0.776471,0.587097,0.769767


Voer nu opnieuw manuele classificatie uit voor de zin "Europe advise money" door gebruik te maken van Laplacian Smoothing.
Is deze zin Spam of Ham?

In [6]:
pTextIsSpam = pSpam * 2/(N_spam + 2* V) * 0.167742 * 0.2
pTextIsHam = pHam * 2/(N_ham + 2* V) * 0.083721 * 0.525581

if(pTextIsSpam > pTextIsHam):
    print("Europe advise money is spam")
else:
    print("Europe advise money is ham")

Europe advise money is ham


**Probleem van veel kansen te vermenigvuldigen**

Naast het probleem van ongeziene woorden, kan er nog een probleem optreden bij het werken met de kansberekeningen.
Aangezien kansen een waarde tussen 0 en 1 zijn kan het zijn dat er bij vermenigvuldiging van veel kansen een floating-point underflow optreed omdat het resultaat steeds kleiner en kleiner gaat worden.
Om dit tegen te gaan kan men gebruik maken van het logaritme van de kansen.
Dit wordt ook **log likelihood** genoemd en wordt berekend als volgt:

$log(P(\text{Spam}|w_1,w_2, \dots w_n)) \propto log(P(\text{Spam})) + \sum\limits_{i=1}^{n}log(P(w_i|\text{Spam}))$

De resulterende klasse is nog steeds de klasse met de hoogste log-likelihood.

Bereken de log-likelihood voor de zin "Europe advice money".

In [7]:
import math

pTextIsSpam = math.log(pSpam) + math.log(2/(N_spam + 2* V)) +  math.log(0.167742) + math.log(0.2)
pTextIsHam = math.log(pHam) + math.log(2/(N_ham + 2* V)) + math.log(0.083721) + math.log(0.525581)

**Probleem van gelijkaardige woorden (Moet nog niet gekend zijn voor de type A evaluatie, komt later terug)**

Daarnaast is het duidelijk dat elke vorm van geschreven tekst een groot aantal overbodige woorden betekenen. 
Dit zijn dan bijvoorbeeld woorden die in heel veel zinnen voorkomen zoals ik, hij, en, daar, ...
Deze woorden gaan geen informatie geven om de classificatie uit te voeren en kunnen dus ook genegeerd worden.
Daarnaast kan men ook de vraag stellen over vervoegingen van werkwoorden of meervouden een apart woord moeten zijn.

Over het algemeen gezien kunnen we de volgende stappen uitvoeren om de tekst bruikbaarder te maken:
* Html/xml/... tags verwijderen indien er zijn ([BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/))
* Cijfers/speciale symbolen verwijderen
* Alles omzetten naar lowercase
* Stopwoorden verwijderen ([nltk.corpus](https://www.geeksforgeeks.org/removing-stop-words-nltk-python/)) (vergeet niet de stopwords te downloaden indien nodig)
* Alle woorden herleiden naar hun stam ([nltk SnowballStemmer](https://www.nltk.org/_modules/nltk/stem/snowball.html))
* Verwijder te korte woorden

Deze stappen kunnen door middel van de volgende lijnen code uitgevoerd worden: